In [10]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader 
import transformers
from transformers import GPT2Tokenizer, GPT2Model
from tqdm import tqdm
import os

In [2]:
data = pd.read_feather('../mini_codenet/data/split/reward_val.ftr')
data.head()

,level_0,index,submission_id,problem_id,language,filename_ext,status,cpu_time,memory,code_size,accuracy,solution,problem_statement
0,761513,7854,s355775556,p03053,C++,cpp,Runtime Error,342.0,5120.0,1831,None,#include <stdio.h>\n#include <stdlib.h>\n#incl...,Score : 300 points \n Problem Statement You ar...
1,761514,15586,s361808912,p03053,Python,py,Runtime Error,1062.0,150772.0,692,None,"H, W = map(int, input().split())\nA = []\nfor ...",Score : 300 points \n Problem Statement You ar...
2,761515,14291,s920384889,p03053,C++,cpp,Runtime Error,389.0,512.0,1372,None,#include <iostream>\nusing namespace std;\n\ni...,Score : 300 points \n Problem Statement You ar...
3,761516,14222,s539766324,p03053,Python,py,Runtime Error,1064.0,138392.0,638,None,# coding:utf-8\n\nif __name__=='__main__':\n ...,Score : 300 points \n Problem Statement You ar...
4,761517,1227,s849699734,p03053,Python,py,Runtime Error,758.0,308244.0,13571,None,"import sys,bisect,string,math,time,functools,r...",Score : 300 points \n Problem Statement You ar...


In [3]:
data.iloc[0]["solution"]

'#include <stdio.h>\n#include <stdlib.h>\n#include <math.h>\n#include <string.h>\n#include <queue>\nusing namespace std;\ntypedef unsigned int uint;\ntypedef long long ll;\ntypedef pair<int, int> P;\nconst int c_INF = 111111111;\nconst int c_mINF = -111111111;\nconst int c_YET = -1;\nconst int c_Dx[4] = { 0,0,1,-1 };\nconst int c_Dy[4] = { 1,-1,0,0 };\n\nvoid debug();\n\n\nint H, W, dst[1000][1000];\nchar  grid[1000][1000];\nvoid BFS(int sy, int sx);\nint main() {\n\tscanf("%d%d", &H, &W);\n\tfor (int i = 0; i < H; i++)scanf("%s", grid[i]);\n\n\tfor (int i = 0; i < H; i++)\n\t\tfor (int j = 0; j < W; j++)dst[i][j] = c_YET;\n\n\tfor (int i = 0; i < H; i++)\n\t\tfor (int j = 0; j < W; j++)\n\t\t\tif (grid[i][j] == \'#\')BFS(i, j);\n\n\tint max = c_mINF;\n\tfor (int i = 0; i < H; i++)\n\t\tfor (int j = 0; j < W; j++)\n\t\t\tif (dst[i][j] > max)max = dst[i][j];\n\n\tprintf("%d", max);\n\n}\n\n\nvoid BFS(int sy,int sx) {\n\tdst[sy][sx] = 0;\n\tqueue<P> que;\n\tque.push(P(sy, sx));\n\n\twhil

In [4]:
class CodeNetDataset(Dataset):
    def __init__(self, feather_path, tokenizer):
        self.data = pd.read_feather(feather_path)
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instance = self.data.iloc[idx]
        # language = self.tokenizer(instance["language"], return_tensors='pt')
        # problem = self.tokenizer(instance["language"] + "::" + instance["problem_statement"], max_length=1028, padding='max_length', truncation=True, return_tensors='pt')
        # code_solution = self.tokenizer(instance["solution"], max_length=1028, padding='max_length', truncation=True, return_tensors='pt')

        problem = self.tokenizer(instance["language"] + "::" + instance["problem_statement"], return_tensors='pt')
        code_solution = self.tokenizer(instance["solution"], return_tensors='pt')

        return instance['problem_id'], problem, code_solution

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2Model.from_pretrained('gpt2-large')

Using pad_token, but it is not set yet.


In [20]:
cnds = CodeNetDataset('../mini_codenet/data/split/pretrain_train.ftr', tokenizer)

In [28]:
n, p, s = cnds[0]
print(n)
print(p.input_ids.shape)
print(s.input_ids.shape)
arr = []
arr.append([n,p.input_ids.shape,s.input_ids.shape])
test_df = pd.DataFrame(arr, columns=['problem_id','input_shape','output_shape'])
test_df.head()

p02936
torch.Size([1, 663])
torch.Size([1, 187])


,problem_id,input_shape,output_shape
0,p02936,"(1, 663)","(1, 187)"


In [11]:
#print(os.listdir('../mini_codenet/data/split/'))

['.gitkeep', 'evaluate_test.ftr', 'evaluate_train.ftr', 'evaluate_val.ftr', 'pretrain_test.ftr', 'pretrain_train.ftr', 'pretrain_val.ftr', 'reward_test.ftr', 'reward_train.ftr', 'reward_val.ftr']


In [29]:
filenames = os.listdir('../mini_codenet/data/split/')
# get all of the encoding lengths for a file
i = 0
for filename in filenames:
    if filename != '.gitkeep':
        cnds = CodeNetDataset(f'../mini_codenet/data/split/{filename}', tokenizer)
        print(i, filename)
        array = []
        #array.append(['problem_id','input_shape','output_shape'])
        for batch in tqdm(cnds):
            n, p, s = batch
            array.append([n, p.input_ids.shape, s.input_ids.shape])
    
        df = pd.DataFrame(array, columns=['problem_id','input_shape','output_shape'])
        print(df.head())
        df.reset_index(inplace=True)
        df.to_feather(f"./encoding_analysis/{filename}")
        i += 1


0 evaluate_test.ftr


100%|██████████| 190378/190378 [15:55<00:00, 199.24it/s]


  problem_id input_shape output_shape
0     p02936    (1, 663)     (1, 187)
1     p02936    (1, 663)     (1, 255)
2     p02936    (1, 663)     (1, 273)
3     p02936    (1, 663)     (1, 340)
4     p02936    (1, 664)     (1, 541)
1 evaluate_train.ftr


100%|██████████| 1332650/1332650 [2:04:54<00:00, 177.81it/s]  


  problem_id input_shape output_shape
0     p02700    (1, 362)     (1, 181)
1     p02700    (1, 362)     (1, 503)
2     p02700    (1, 361)     (1, 110)
3     p02700    (1, 362)     (1, 815)
4     p02700    (1, 362)    (1, 1554)
2 evaluate_val.ftr


...

  problem_id input_shape output_shape
0     p03916    (1, 651)    (1, 1100)
1     p03916    (1, 651)    (1, 1580)
2     p03916    (1, 651)     (1, 960)
3     p03916    (1, 651)     (1, 576)
4     p03916    (1, 651)     (1, 382)
7 reward_train.ftr


100%|██████████| 666324/666324 [59:12<00:00, 187.55it/s]  


  problem_id input_shape output_shape
0     p03644    (1, 305)      (1, 59)
1     p03644    (1, 305)      (1, 78)
2     p03644    (1, 305)     (1, 100)
3     p03644    (1, 305)     (1, 159)
4     p03644    (1, 305)     (1, 235)
8 reward_val.ftr


100%|██████████| 190379/190379 [18:02<00:00, 175.92it/s]


  problem_id input_shape output_shape
0     p03053    (1, 371)     (1, 999)
1     p03053    (1, 370)     (1, 393)
2     p03053    (1, 371)    (1, 1022)
3     p03053    (1, 370)     (1, 372)
4     p03053    (1, 370)    (1, 7815)


In [21]:
# get the max len of all tokenizations
# this cell takes around 20 minutes
maxlen_i = 0
maxlen_j = 0
for i, j in tqdm(cnds):
    bi, wi = i.input_ids.shape
    bj, wj = j.input_ids.shape

    if wi > maxlen_i:
        maxlen_i = wi

    if wj > maxlen_j:
        maxlen_j = wj

print(maxlen_i)
print(maxlen_j)

100%|██████████| 190378/190378 [18:58<00:00, 167.22it/s]  

1465
293127


In [51]:

dl = DataLoader(cnds, batch_size=10, shuffle=True)
for i, batch in enumerate(dl):
    print(i, batch[1].input_ids.shape)
    break

0 torch.Size([10, 1, 1028])
